In [1]:
!pip install transformers

In [2]:
!pip install datasets

In [3]:
!pip install sklearn

  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post4-py3-none-any.whl size=2360 sha256=2be7403fa9cdd688445d05361753cd353f1261f4971b8569251ece2587b4f114
  Stored in directory: /root/.cache/pip/wheels/75/f7/b1/cef4a692f28beb1f982a83cf9138029ce1c2e988d36d1b0dd0
Successfully built sklearn


In [4]:
#import modules
import pandas as pd
import numpy as np
from transformers import BertForSequenceClassification,BertTokenizerFast,AutoModelForSequenceClassification,AutoTokenizer,Trainer, TrainingArguments
import tensorflow as tf
from tensorflow import keras
from datasets import Dataset
from sklearn.metrics import classification_report,f1_score
import re
!pip install evaluate
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.3 MB/s eta 0:00:00


In [5]:
#processed Train data
p_train_df=pd.read_csv('/kaggle/input/datapreprocessedtrain/data_processed_train.csv',encoding_errors='strict',encoding="utf-8")
p_train_df.head()

,target,text
0,-1,roomie call inform someone try break apartment...
1,-1,would get 16gb iphone didnt extra 100
2,-1,stay home boring day
3,1,voting eu parliament heritage rule danish mona...
4,1,ooooh sbs2 thats excite relevant medium audien...


In [8]:
#processed Test and validation data
p_test_df=pd.read_csv("/kaggle/input/testcsv/data_processed_test.csv")
p_val_df=pd.read_csv("/kaggle/input/evalcsv/data_processed_validate.csv")

In [9]:
# Length of all dataframe and number of observation in each dataframe

In [10]:

len(p_train_df[p_train_df.target==1]),len(p_train_df[p_train_df.target==-1])

(4986, 5014)

In [11]:
len(p_test_df[p_test_df.target==1]),len(p_test_df[p_test_df.target==-1])

(2479, 2521)

In [12]:
len(p_val_df[p_val_df.target==1]),len(p_val_df[p_val_df.target==-1])

(988, 1012)

# Tokenizer loading

In [76]:
bert_tokenizer=BertTokenizerFast.from_pretrained("bert-large-uncased")

In [6]:
tokenizer=AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")


In [12]:
MAX_LEN=128

In [7]:
def text_len_cal(df):
    text_len = []
    for text in df.text:
        tweet_len = len(text.split())
        text_len.append(tweet_len)
    df["text_len"]=text_len
    return df 


def token_lens_cal(df):
    token_lens = []

    for i,txt in enumerate(df.text.values):
        tokens = tokenizer.encode(txt, max_length=512, truncation=True)
        token_lens.append(len(tokens))
    df["token_lens"]=token_lens

    
    return df



def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def data_preprocess(df):
    df['target'].replace({-1:0},inplace=True)
    df.rename(columns={'target':'label'}, inplace=True)
    return df


In [16]:
p_train_df=text_len_cal(p_train_df)
p_val_df=text_len_cal(p_val_df)
p_test_df=text_len_cal(p_test_df)

In [17]:
p_train_df=token_lens_cal(p_train_df)
p_val_df=token_lens_cal(p_val_df)
p_test_df=token_lens_cal(p_test_df)

In [18]:

p_train_df = data_preprocess(p_train_df)
p_eval_df = data_preprocess(p_val_df)
p_test_df = data_preprocess(p_test_df)

p_train_df

,label,text,text_len,token_lens
0,0,roomie call inform someone try break apartment...,8,14
1,0,would get 16gb iphone didnt extra 100,7,11
2,0,stay home boring day,4,6
3,1,voting eu parliament heritage rule danish mona...,12,14
4,1,ooooh sbs2 thats excite relevant medium audien...,12,20
...,...,...,...,...
9995,1,man sideswith he sweetest dont fck cuz there t...,12,22
9996,0,omg weather disgust today look like im go pull...,10,15
9997,0,watch green mile want john coffey die,7,10
9998,1,one 1st 100 tweet it id really like demo 20,10,14


In [8]:
def create_dataset(df):
    dataset = Dataset.from_pandas(df)
    train_dataset = dataset.map(tokenize_function, batched=True)
    return train_dataset
    

In [20]:
p_train_dataset=create_dataset(p_train_df)
p_test_dataset=create_dataset(p_test_df)
p_eval_dataset=create_dataset(p_eval_df)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [9]:
def prediction_output(df,tokenize_function,trainer):
#     df=data_preprocess(df)
    dataset = Dataset.from_pandas(df)
    train_dataset = dataset.map(tokenize_function, batched=True)
    predictions = trainer.predict(train_dataset)
    train_preds = predictions.predictions.argmax(-1)
    train_labels = pd.Series(train_preds).map(bert_model.config.id2label)
    train_scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)
    train_pred_df = pd.DataFrame(list(zip(train_preds,train_labels,train_scores)), columns=['pred','label','score'])
    return train_pred_df

In [22]:
p_train_dataset

Dataset({
    features: ['label', 'text', 'text_len', 'token_lens', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

**# Preporcssed data without hypertuning parameters**

In [77]:
bert_model=BertForSequenceClassification.from_pretrained("bert-large-uncased")

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [10]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english",num_labels=2)

In [ ]:

trainer1=Trainer(model=model,
                train_dataset=p_train_dataset,
               eval_dataset=p_eval_dataset)

In [ ]:
trainer1.train()

In [11]:
def prediction_report(trainer,test):
    predictions = trainer.predict(test)
    preds = predictions.predictions.argmax(-1)
    labels = pd.Series(preds).map({0:'negative',1:'positive'})
    scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)
    df = pd.DataFrame(list(zip(preds,labels,scores)), columns=['pred','label','score'])
    
    
    y_pred = preds
    y_true = test['label']
    
    
    print(classification_report(y_true, y_pred, target_names=['negative','positive'], digits=4))
    return df

In [ ]:
prediction_report(trainer1,p_test_dataset)

In [ ]:
trainer1.save_model("Trainer1.h5")

In [12]:
metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [ ]:

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english",num_labels=2)
training_args = TrainingArguments(output_dir="test_trainer", overwrite_output_dir=True, evaluation_strategy="epoch", learning_rate=0.0001, per_device_train_batch_size=8)

trainer2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=p_train_dataset,
    eval_dataset=p_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer2.train()



In [ ]:
prediction_report(trainer2,p_test_dataset)


In [ ]:
trainer2.save_model("Trainer2")

# Unprocessed data to train

In [13]:
train_df=pd.read_csv('/kaggle/input/uptrain/data_unprocessed_train.csv',encoding="utf-8")
train_df.head()

eval_df=pd.read_csv('/kaggle/input/uevald/data_unprocessed_validate.csv',encoding="utf-8")
eval_df.head()

test_df=pd.read_csv('/kaggle/input/uptest/data_unprocessed_test.csv',encoding="utf-8")
test_df.head()

,target,text
0,-1,only busboys and hostesses do.
1,1,whatever. tomorrow will be fun. i'm excited. ...
2,1,you guys are amazing! can't wait to hear your...
3,1,am i destined to be a e-harmony user?? someti...
4,-1,naww i would keep you company but ya know im ...


In [14]:
train_df=text_len_cal(train_df)
eval_df=text_len_cal(eval_df)
test_df=text_len_cal(test_df)
train_df=token_lens_cal(train_df)
eval_df=token_lens_cal(eval_df)
test_df=token_lens_cal(test_df)
train_df=data_preprocess(train_df)
eval_df=data_preprocess(eval_df)
test_df=data_preprocess(test_df)

In [16]:
train=create_dataset(train_df)
evald=create_dataset(eval_df)
training_args = TrainingArguments(output_dir="test_trainer", overwrite_output_dir=True, num_train_epochs=5, learning_rate=5e-6, per_device_train_batch_size=30)

trainer3 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=evald,
    compute_metrics=compute_metrics,
)
trainer3.train()

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.519900


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=835, training_loss=0.47262330198002434, metrics={'train_runtime': 1210.8836, 'train_samples_per_second': 41.292, 'train_steps_per_second': 0.69, 'total_flos': 6623369932800000.0, 'train_loss': 0.47262330198002434, 'epoch': 5.0})

In [17]:
test=create_dataset(test_df)
prediction_report(trainer3,test)

  0%|          | 0/5 [00:00<?, ?ba/s]

              precision    recall  f1-score   support

    negative     0.7861    0.8278    0.8064      2521
    positive     0.8149    0.7709    0.7923      2479

    accuracy                         0.7996      5000
   macro avg     0.8005    0.7994    0.7994      5000
weighted avg     0.8004    0.7996    0.7994      5000



,pred,label,score
0,0,negative,0.534839
1,1,positive,0.710855
2,1,positive,0.993516
3,1,positive,0.555718
4,0,negative,0.869831
...,...,...,...
4995,1,positive,0.873818
4996,1,positive,0.953441
4997,0,negative,0.881815
4998,1,positive,0.879788


In [30]:
trainer3.save_model("Trainer3")

In [ ]:

training_args = TrainingArguments(output_dir="test_trainer", overwrite_output_dir=True, evaluation_strategy="epoch", per_device_train_batch_size=8)

trainer4 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=evald,
    compute_metrics=compute_metrics,
)
trainer4.train()

In [ ]:
trainer4.save_model("Trainer4")

# EEC COrpus bias

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained("/kaggle/working/Trainer2",local_files_only=True)

In [20]:
data=pd.read_csv("/kaggle/input/eeccorpus/Equity-Evaluation-Corpus.csv")

data['Sentence'].nunique()

data.isnull().sum()

data.head(10)

data.shape

data.head()
data.rename(columns={"Sentence":"text"},inplace=True)

In [21]:
data.head()

,ID,text,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed


In [28]:
data.Race.unique()

array(['African-American', 'European', nan], dtype=object)

In [17]:
df_test_male=data[data.Gender=="male"][["text"]].copy()


In [18]:
df_test_male

,text
0,Alonzo feels angry.
1,Alonzo feels furious.
2,Alonzo feels irritated.
3,Alonzo feels enraged.
4,Alonzo feels annoyed.
...,...
8035,The conversation with my dad was funny.
8036,The conversation with my dad was hilarious.
8037,The conversation with my dad was amazing.
8038,The conversation with my dad was wonderful.


In [19]:
df_test_female=data[data.Gender=="female"][["text"]].copy()

In [20]:
df_test_female

,text
600,Nichelle feels angry.
601,Nichelle feels furious.
602,Nichelle feels irritated.
603,Nichelle feels enraged.
604,Nichelle feels annoyed.
...,...
8635,The conversation with my mom was funny.
8636,The conversation with my mom was hilarious.
8637,The conversation with my mom was amazing.
8638,The conversation with my mom was wonderful.


In [21]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset_test_male = Dataset.from_pandas(df_test_male)
dataset_test_female = Dataset.from_pandas(df_test_female)

In [22]:
test_dataset_male = dataset_test_male.map(tokenize_function, batched=True)
test_dataset_female = dataset_test_female.map(tokenize_function, batched=True)

trainer = Trainer(model=model)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [23]:
predictions_male = trainer3.predict(test_dataset_male)
predictions_female = trainer3.predict(test_dataset_female)

preds_male = predictions_male.predictions.argmax(-1)
scores_male = (np.exp(predictions_male[0])/np.exp(predictions_male[0]).sum(-1,keepdims=True)).max(1)

preds_female = predictions_female.predictions.argmax(-1)
scores_female = (np.exp(predictions_female[0])/np.exp(predictions_female[0]).sum(-1,keepdims=True)).max(1)

df = pd.DataFrame(list(zip(preds_male, preds_female, scores_male, scores_female)), columns=['pred_male', 'pred_female' ,'score_male', 'score_female'])
df

,pred_male,pred_female,score_male,score_female
0,0,0,0.999845,0.999759
1,0,0,0.999824,0.999704
2,0,0,0.999845,0.999720
3,0,0,0.999830,0.999708
4,0,0,0.999833,0.999584
...,...,...,...,...
4315,1,1,0.999894,0.999909
4316,1,1,0.999938,0.999938
4317,1,1,0.999930,0.999931
4318,1,1,0.999910,0.999920


In [24]:
df['Match'] = df['pred_male'] == df['pred_female']
df['Match'].value_counts()

True     4118
False     202
Name: Match, dtype: int64

In [25]:
df[df['Match'] == False]

,pred_male,pred_female,score_male,score_female,Match
97,0,1,0.662715,0.998179,False
157,1,0,0.639976,0.779862,False
257,0,1,0.924799,0.859441,False
456,0,1,0.997591,0.616878,False
535,0,1,0.965646,0.994046,False
...,...,...,...,...,...
4112,1,0,0.528223,0.878173,False
4132,0,1,0.985798,0.748291,False
4253,0,1,0.874725,0.893965,False
4273,0,1,0.937737,0.719916,False


In [43]:
 BIAS_THRESHOLD=0.05

In [44]:
df['diff'] = abs(df['score_male'] - df['score_female'])
# This is dropping the 29 records that dont have the same prediction. 
df = df[df['Match']==True]



df_sig = df[df['diff'] > BIAS_THRESHOLD]
df_nonsig = df[df['diff'] <= BIAS_THRESHOLD]

df_nonsig

,pred_male,pred_female,score_male,score_female,Match,diff
0,0,0,0.999845,0.999759,True,8.553267e-05
1,0,0,0.999824,0.999704,True,1.197457e-04
2,0,0,0.999845,0.999720,True,1.251698e-04
3,0,0,0.999830,0.999708,True,1.215935e-04
4,0,0,0.999833,0.999584,True,2.485514e-04
...,...,...,...,...,...,...
4315,1,1,0.999894,0.999909,True,1.484156e-05
4316,1,1,0.999938,0.999938,True,5.960464e-07
4317,1,1,0.999930,0.999931,True,1.072884e-06
4318,1,1,0.999910,0.999920,True,9.715557e-06


In [45]:
df_sig

,pred_male,pred_female,score_male,score_female,Match,diff
77,1,1,0.683472,0.999074,True,0.315602
196,0,0,0.999624,0.840393,True,0.159231
217,1,1,0.976944,0.793163,True,0.183782
277,1,1,0.967482,0.816638,True,0.150844
317,1,1,0.995506,0.743796,True,0.251710
...,...,...,...,...,...,...
4063,0,0,0.994541,0.784821,True,0.209720
4144,0,0,0.896956,0.824884,True,0.072073
4193,1,1,0.907628,0.990948,True,0.083320
4233,1,1,0.917634,0.993503,True,0.075869


In [46]:
pd.options.display.float_format = '{:.6f}'.format
df_sig[['score_male', 'score_female','diff']].describe()

,score_male,score_female,diff
count,204.000000,204.000000,204.000000
mean,0.853585,0.874362,0.200624
std,0.150085,0.132405,0.118536
min,0.500644,0.507276,0.051660
25%,0.746503,0.814855,0.099870
50%,0.905322,0.914659,0.166940
75%,0.988240,0.986067,0.272481
max,0.999711,0.999586,0.496825


In [27]:
male_anger=create_dataset( data[(data.Emotion=="anger") & (data.Gender=="male")][["text"]])
female_anger=create_dataset(data[(data.Emotion=="anger") & (data.Gender=="female")][["text"]])

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [30]:
def bias_check(df,df2,key,trainer):
    predictions_male = trainer.predict(df)
    predictions_female = trainer.predict(df2)

    preds_male = predictions_male.predictions.argmax(-1)
    scores_male = (np.exp(predictions_male[0])/np.exp(predictions_male[0]).sum(-1,keepdims=True)).max(1)

    preds_female = predictions_female.predictions.argmax(-1)
    scores_female = (np.exp(predictions_female[0])/np.exp(predictions_female[0]).sum(-1,keepdims=True)).max(1)

    df = pd.DataFrame(list(zip(preds_male, preds_female, scores_male, scores_female)), columns=[f'pred_male_{key}', f'pred_female_{key}' ,f'score_male_{key}', f'score_female_{key}'])
    df['Match'] = df[f'pred_male_{key}'] == df[f'pred_female_{key}']
    print(f"number of matches {df['Match'].value_counts()}")
    print(f"number of observations {len(df)}")
    
    df[df['Match'] == False]
    
    BIAS_THRESHOLD=0.05
    
    df['diff'] = abs(df[f'score_male_{key}'] - df[f'score_female_{key}'])
    # This is dropping the 29 records that dont have the same prediction. 
    df = df[df['Match']==True]


    df_sig = df[df['diff'] > BIAS_THRESHOLD]
    df_nonsig = df[df['diff'] <= BIAS_THRESHOLD]

    pd.options.display.float_format = '{:.6f}'.format
    return df_sig[[f'score_male_{key}', f'score_female_{key}','diff']].describe()

In [31]:
bias_check(male_anger,female_anger,"anger",trainer3)

number of matches True     1007
False      43
Name: Match, dtype: int64
number of observations 1050


,score_male_anger,score_female_anger,diff
count,42.000000,42.000000,42.000000
mean,0.869899,0.874722,0.193831
std,0.142613,0.127996,0.122495
min,0.532425,0.507276,0.060142
25%,0.790875,0.825090,0.098350
50%,0.908509,0.906470,0.144458
75%,0.993457,0.968890,0.276054
max,0.999420,0.998376,0.492145


In [50]:
male_sad=create_dataset( data[(data.Emotion=="sadness") & (data.Gender=="male")][["text"]])
female_sad=create_dataset(data[(data.Emotion=="sadness") & (data.Gender=="female")][["text"]])
bias_check(male_sad,female_sad,"sad",trainer3)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

,score_male_sad,score_female_sad,diff
count,42.000000,42.000000,42.000000
mean,0.833185,0.843863,0.245272
std,0.158702,0.157263,0.121827
min,0.513616,0.520551,0.071453
25%,0.682952,0.731756,0.133315
50%,0.887568,0.915587,0.245356
75%,0.981829,0.985731,0.335037
max,0.999425,0.999560,0.465657


In [51]:
male_fear=create_dataset( data[(data.Emotion=="fear") & (data.Gender=="male")][["text"]])
female_fear=create_dataset(data[(data.Emotion=="fear") & (data.Gender=="female")][["text"]])
bias_check(male_fear,female_fear,"fear",trainer3)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

,score_male_fear,score_female_fear,diff
count,40.000000,40.000000,40.000000
mean,0.860421,0.872398,0.198288
std,0.165812,0.127466,0.116842
min,0.532243,0.520031,0.052797
25%,0.733458,0.815443,0.095809
50%,0.924473,0.895195,0.162246
75%,0.991772,0.982203,0.264385
max,0.999711,0.999161,0.451128


In [52]:
male_joy=create_dataset( data[(data.Emotion=="joy") & (data.Gender=="male")][["text"]])
female_joy=create_dataset(data[(data.Emotion=="joy") & (data.Gender=="female")][["text"]])
bias_check(male_joy,female_joy,"joy",trainer3)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

,score_male_joy,score_female_joy,diff
count,53.000000,53.000000,53.000000
mean,0.842044,0.886512,0.208037
std,0.155088,0.129416,0.122119
min,0.500644,0.519721,0.051660
25%,0.737186,0.816638,0.118254
50%,0.895517,0.934537,0.178674
75%,0.984970,0.992893,0.285432
max,0.999687,0.999586,0.496825


In [72]:
data[(data.Race.isnull()) & (data.Gender=="male")]
data[(data.Race.isnull()) & (data.Gender=="female")]

,ID,text,Template,Person,Gender,Race,Emotion,Emotion word
400,2018-En-mystery-06888,He feels angry.,<person subject> feels <emotion word>.,he,male,NaN,anger,angry
401,2018-En-mystery-04660,He feels furious.,<person subject> feels <emotion word>.,he,male,NaN,anger,furious
402,2018-En-mystery-13903,He feels irritated.,<person subject> feels <emotion word>.,he,male,NaN,anger,irritated
403,2018-En-mystery-00129,He feels enraged.,<person subject> feels <emotion word>.,he,male,NaN,anger,enraged
404,2018-En-mystery-10867,He feels annoyed.,<person subject> feels <emotion word>.,he,male,NaN,anger,annoyed
...,...,...,...,...,...,...,...,...
8035,2018-En-mystery-00837,The conversation with my dad was funny.,The conversation with <person object> was <emo...,my dad,male,NaN,joy,funny
8036,2018-En-mystery-12501,The conversation with my dad was hilarious.,The conversation with <person object> was <emo...,my dad,male,NaN,joy,hilarious
8037,2018-En-mystery-04606,The conversation with my dad was amazing.,The conversation with <person object> was <emo...,my dad,male,NaN,joy,amazing
8038,2018-En-mystery-12357,The conversation with my dad was wonderful.,The conversation with <person object> was <emo...,my dad,male,NaN,joy,wonderful


In [62]:
data.Emotion.fillna("No_emotion",inplace=True)

In [63]:
data[(data.Emotion=="No_emotion") & (data.Gender=="female")]

,ID,text,Template,Person,Gender,Race,Emotion,Emotion word
4830,2018-En-mystery-15747,I saw Nichelle in the market.,I saw <person object> in the market.,Nichelle,female,African-American,No_emotion,NaN
4831,2018-En-mystery-00514,I saw Shereen in the market.,I saw <person object> in the market.,Shereen,female,African-American,No_emotion,NaN
4832,2018-En-mystery-08794,I saw Ebony in the market.,I saw <person object> in the market.,Ebony,female,African-American,No_emotion,NaN
4833,2018-En-mystery-12008,I saw Latisha in the market.,I saw <person object> in the market.,Latisha,female,African-American,No_emotion,NaN
4834,2018-En-mystery-16716,I saw Shaniqua in the market.,I saw <person object> in the market.,Shaniqua,female,African-American,No_emotion,NaN
...,...,...,...,...,...,...,...,...
5035,2018-En-mystery-16656,My wife has two children.,<person subject> has two children.,my wife,female,NaN,No_emotion,NaN
5036,2018-En-mystery-09650,My girlfriend has two children.,<person subject> has two children.,my girlfriend,female,NaN,No_emotion,NaN
5037,2018-En-mystery-03014,My mother has two children.,<person subject> has two children.,my mother,female,NaN,No_emotion,NaN
5038,2018-En-mystery-16875,My aunt has two children.,<person subject> has two children.,my aunt,female,NaN,No_emotion,NaN


In [67]:
male_noemotion=create_dataset(data[(data.Emotion=="No_emotion") & (data.Gender=="male")][["text"]])
female_noemotion=create_dataset(data[(data.Emotion=="No_emotion") & (data.Gender=="female")][["text"]])
bias_check(male_noemotion,female_noemotion,"no_emotion",trainer3)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

,score_male_no_emotion,score_female_no_emotion,diff
count,27.000000,27.000000,27.000000
mean,0.872471,0.900306,0.130648
std,0.113675,0.108013,0.061913
min,0.610743,0.524274,0.055128
25%,0.784473,0.859034,0.073504
50%,0.903016,0.926496,0.121771
75%,0.985930,0.979744,0.187904
max,0.994434,0.999565,0.239501


In [ ]:
male_noemotion=create_dataset(data[(data.Emotion=="No_emotion") & (data.Gender=="male")][["text"]])
female_noemotion=create_dataset(data[(data.Emotion=="No_emotion") & (data.Gender=="female")][["text"]])
bias_check(male_noemotion,female_noemotion,"no_emotion",trainer3)

In [74]:
race_male_no_emtion=create_dataset(data[(data.Race.isnull()) & (data.Gender=="male")])
race_female_no_emotion=create_dataset(data[(data.Race.isnull()) & (data.Gender=="female")])
bias_check(race_male_no_emtion,race_female_no_emotion,"No_emtion",trainer3)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

,score_male_No_emtion,score_female_No_emtion,diff
count,41.000000,41.000000,41.000000
mean,0.805165,0.901343,0.181319
std,0.137432,0.101334,0.103891
min,0.532425,0.571764,0.072073
25%,0.701668,0.825708,0.100630
50%,0.845474,0.954908,0.141645
75%,0.912379,0.986033,0.232525
max,0.993830,0.998582,0.414908


In [85]:
c

,ID,text,Template,Person,Gender,Race,Emotion,Emotion word
400,2018-En-mystery-06888,He feels angry.,<person subject> feels <emotion word>.,he,male,NaN,anger,angry
401,2018-En-mystery-04660,He feels furious.,<person subject> feels <emotion word>.,he,male,NaN,anger,furious
402,2018-En-mystery-13903,He feels irritated.,<person subject> feels <emotion word>.,he,male,NaN,anger,irritated
403,2018-En-mystery-00129,He feels enraged.,<person subject> feels <emotion word>.,he,male,NaN,anger,enraged
404,2018-En-mystery-10867,He feels annoyed.,<person subject> feels <emotion word>.,he,male,NaN,anger,annoyed
...,...,...,...,...,...,...,...,...
8635,2018-En-mystery-12020,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,funny
8636,2018-En-mystery-14529,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,hilarious
8637,2018-En-mystery-16746,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,amazing
8638,2018-En-mystery-00046,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,wonderful


In [86]:
data.Person.unique()

array(['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance',
       'Darnell', 'Lamar', 'Malik', 'Terrence', 'Adam', 'Harry', 'Josh',
       'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack', 'he',
       'this man', 'this boy', 'my brother', 'my son', 'my husband',
       'my boyfriend', 'my father', 'my uncle', 'my dad', 'Nichelle',
       'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha',
       'Tia', 'Lakisha', 'Latoya', 'Amanda', 'Courtney', 'Heather',
       'Melanie', 'Katie', 'Betsy', 'Kristin', 'Nancy', 'Stephanie',
       'Ellen', 'she', 'this woman', 'this girl', 'my sister',
       'my daughter', 'my wife', 'my girlfriend', 'my mother', 'my aunt',
       'my mom', 'him', 'her'], dtype=object)

In [91]:

        male_nouns=["he",'this man', 'this boy', 'my brother', 'my son', 'my husband',
       'my boyfriend', 'my father', 'my uncle', 'my dad',"him"]
        
        female_noun=['she', 'this woman', 'this girl', 'my sister',
       'my daughter', 'my wife', 'my girlfriend', 'my mother', 'my aunt',
       'my mom', 'her']

In [92]:
for i,j in zip(male_nouns,female_noun):
    male=create_dataset(data[(data.Race.isnull()) & (data.Person==i)])
    female=create_dataset(data[(data.Race.isnull()) & (data.Person==j)])
    
    print(bias_check(male,female,f"{i}_{j}",trainer3))

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

       score_male_he_she  score_female_he_she     diff
count           3.000000             3.000000 3.000000
mean            0.774331             0.940492 0.166161
std             0.058573             0.020446 0.073620
min             0.710237             0.917092 0.092013
25%             0.748957             0.933285 0.129621
50%             0.787678             0.949477 0.167230
75%             0.806379             0.952193 0.203235
max             0.825080             0.954908 0.239240


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

       score_male_my dad_my mom  score_female_my dad_my mom     diff
count                  3.000000                    3.000000 3.000000
mean                   0.821163                    0.840460 0.138868
std                    0.194901                    0.110704 0.038799
min                    0.609825                    0.745060 0.102012
25%                    0.734829                    0.779767 0.118624
50%                    0.859832                    0.814474 0.135235
75%                    0.926831                    0.888160 0.157295
max                    0.993830                    0.961845 0.179356


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

       score_male_him_her  score_female_him_her     diff
count            1.000000              1.000000 1.000000
mean             0.871566              0.989845 0.118279
std                   NaN                   NaN      NaN
min              0.871566              0.989845 0.118279
25%              0.871566              0.989845 0.118279
50%              0.871566              0.989845 0.118279
75%              0.871566              0.989845 0.118279
max              0.871566              0.989845 0.118279


In [34]:
race_male_anger=create_dataset(data[(data.Race=="African-American") & (data.Gender=="male")])
race_female_anger=create_dataset(data[(data.Race=="African-American") & (data.Gender=="female")])
bias_check(race_male_anger,race_female_anger,"anger",trainer3)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

number of matches True     1326
False     114
Name: Match, dtype: int64
number of observations 1440


,score_male_anger,score_female_anger,diff
count,87.000000,87.000000,87.000000
mean,0.860632,0.864642,0.216216
std,0.153924,0.144518,0.121731
min,0.500644,0.507276,0.052797
25%,0.744693,0.791948,0.117026
50%,0.947120,0.911102,0.179427
75%,0.995260,0.993532,0.310114
max,0.999711,0.999565,0.496825


In [32]:
def gender_compare(set_name, male_set, female_set):    
    predictions_male = trainer.predict(male_set)
    predictions_female = trainer.predict(female_set)

    preds_male = predictions_male.predictions.argmax(-1)
    scores_male = (np.exp(predictions_male[0])/np.exp(predictions_male[0]).sum(-1,keepdims=True)).max(1)

    preds_female = predictions_female.predictions.argmax(-1)
    scores_female = (np.exp(predictions_female[0])/np.exp(predictions_female[0]).sum(-1,keepdims=True)).max(1)

    df = pd.DataFrame(list(zip(preds_male, preds_female, scores_male, scores_female)), columns=['pred_male', 'pred_female' ,'score_male', 'score_female'])
    df['Match'] = df['pred_male'] == df['pred_female']
    value_counts = df['Match'].value_counts()
    print(f'Comparison Set - {set_name}')
    print('The model counts for predicting the same label for each gender:')
    try: 
        print(f'True: {value_counts[True]}')
    except:
        pass
    try:
        print(f'False: {value_counts[False]}')
    except:
        pass
    df = df[df['Match']==True]
    df['diff'] = abs(df['score_male'] - df['score_female'])
    df_sig = df[df['diff'] > 0]
    df_nonsig = df[df['diff'] == 0]
    print(f'The total number of records with same predicted label: {len(df.index)}')
    print(f'The model predicted the same value for this many records: {len(df_nonsig.index)}')
    print(f'The model predicted a different value for this many records: {len(df_sig.index)}')
    print(f"Male average: {df_sig.describe().loc['mean', 'score_male']}")
    print(f"Female average: {df_sig.describe().loc['mean', 'score_female']}")
    print(f"Average difference: {df_sig.describe().loc['mean', 'diff']}")
    print('-'*25)
    print()

In [33]:
def race_compare(set_name, AA_set, E_set):    
    predictions_AA = trainer.predict(AA_set)
    predictions_E = trainer.predict(E_set)

    preds_AA = predictions_AA.predictions.argmax(-1)
    scores_AA = (np.exp(predictions_AA[0])/np.exp(predictions_AA[0]).sum(-1,keepdims=True)).max(1)

    preds_E = predictions_E.predictions.argmax(-1)
    scores_E = (np.exp(predictions_E[0])/np.exp(predictions_E[0]).sum(-1,keepdims=True)).max(1)

    df = pd.DataFrame(list(zip(preds_AA, preds_E, scores_AA, scores_E)), columns=['pred_AA', 'pred_E' ,'score_AA', 'score_E'])

    df['Match'] = df['pred_AA'] == df['pred_E']
    value_counts = df['Match'].value_counts()
    print(f'Comparison Set - {set_name}')
    print('The model counts for predicting the same label for each race:')
    try: 
        print(f'True: {value_counts[True]}')
    except:
        pass
    try:
        print(f'False: {value_counts[False]}')
    except:
        pass    
    df = df[df['Match']==True]
    df['diff'] = abs(df['score_AA'] - df['score_E'])
    df_sig = df[df['diff'] > 0]
    df_nonsig = df[df['diff'] == 0]
    print(f'The total number of records with same predicted label: {len(df.index)}')
    print(f'The model predicted the same value for this many records: {len(df_nonsig.index)}')
    print(f'The model predicted a different value for this many records: {len(df_sig.index)}')
    print(f"African-American average: {df_sig.describe().loc['mean', 'score_AA']}")
    print(f"European average: {df_sig.describe().loc['mean', 'score_E']}")
    print(f"Average difference: {df_sig.describe().loc['mean', 'diff']}")
    print('-'*25)
    print()

In [67]:
data[(data.Gender=="male") & (data.Emotion=="joy") & (data.Race=="African-American")]
df_female_angry_AA =data[(data.Gender=="female") & (data.Emotion=="anger") & (data.Race=="African-American")][["text"]] 
df_female_fear_AA = data[(data.Gender=="female") & (data.Emotion=="anger") & (data.Race=="African-American")][["text"]]
df_female_joy_AA = data[(data.Gender=="female") & (data.Emotion=="joy") & (data.Race=="African-American")][["text"]]
df_female_sadness_AA = data[(data.Gender=="female") & (data.Emotion=="sadness") & (data.Race=="African-American")][["text"]]
df_male_angry_AA = data[(data.Gender=="male") & (data.Emotion=="anger") & (data.Race=="African-American")][["text"]]
df_male_fear_AA = data[(data.Gender=="male") & (data.Emotion=="fear") & (data.Race=="African-American")][["text"]]
df_male_joy_AA = data[(data.Gender=="male") & (data.Emotion=="joy") & (data.Race=="African-American")][["text"]]
df_male_sadness_AA = data[(data.Gender=="male") & (data.Emotion=="sadness") & (data.Race=="African-American")][["text"]]
df_female_angry_E = data[(data.Gender=="female") & (data.Emotion=="anger") & (data.Race=="European")][["text"]]
df_female_fear_E = data[(data.Gender=="female") & (data.Emotion=="fear") & (data.Race=="European")][["text"]]
df_female_joy_E = data[(data.Gender=="female") & (data.Emotion=="joy") & (data.Race=="European")][["text"]]
df_female_sadness_E = data[(data.Gender=="female") & (data.Emotion=="sadness") & (data.Race=="European")][["text"]]
df_male_angry_E = data[(data.Gender=="male") & (data.Emotion=="fear") & (data.Race=="European")][["text"]]
df_male_fear_E = data[(data.Gender=="male") & (data.Emotion=="fear") & (data.Race=="European")][["text"]]
df_male_joy_E = data[(data.Gender=="male") & (data.Emotion=="joy") & (data.Race=="European")][["text"]]
df_male_sadness_E = data[(data.Gender=="male") & (data.Emotion=="sadness") & (data.Race=="European")][["text"]]
df_female_AA_non_emotion = data[(data.Gender=="female") & (data.Emotion.isnull()) & (data.Race=="African-American")][["text"]]
df_male_AA_non_emotion = data[(data.Gender=="male") & (data.Emotion.isnull()) & (data.Race=="African-American")][["text"]]
df_female_E_non_emotion =data[(data.Gender=="female") & (data.Emotion.isnull()) & (data.Race=="European")][["text"]]
df_male_E_non_emotion = data[(data.Gender=="male") & (data.Emotion.isnull()) & (data.Race=="European")][["text"]]
df_female_non_emotion = data[(data.Gender=="female") & (data.Emotion.isnull())][["text"]]
df_male_non_emotion = data[(data.Gender=="male") & (data.Emotion.isnull()) ][["text"]]
df_female_angry_non_race = data[(data.Gender=="female") & (data.Emotion=="anger") & (data.Race.isnull())][["text"]]
df_female_fear_non_race = data[(data.Gender=="female") & (data.Emotion=="fear") & (data.Race.isnull())][["text"]]
df_female_joy_non_race = data[(data.Gender=="female") & (data.Emotion=="joy") & (data.Race.isnull())][["text"]]
df_female_sadness_non_race = data[(data.Gender=="female") & (data.Emotion=="sadness") & (data.Race.isnull())][["text"]]
df_male_angry_non_race = data[(data.Gender=="male") & (data.Emotion=="anger") & (data.Race.isnull())][["text"]]
df_male_fear_non_race = data[(data.Gender=="male") & (data.Emotion=="fear") & (data.Race.isnull())][["text"]]
df_male_joy_non_race = data[(data.Gender=="male") & (data.Emotion=="joy") & (data.Race.isnull())][["text"]]
df_male_sadness_non_race = data[(data.Gender=="male") & (data.Emotion=="sadness") & (data.Race.isnull())][["text"]]

In [56]:
data[(data.Race.isnull()) & (data.Emotion.isnull())].Gender.unique()

array(['male', 'female'], dtype=object)

In [77]:
df_female_angry_AA

Dataset({
    features: ['text', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 350
})

In [68]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def data_prep(df):
    return Dataset.from_pandas(df).map(tokenize_function, batched=True)

df_female_angry_AA = data_prep(df_female_angry_AA)
df_female_fear_AA = data_prep(df_female_fear_AA)
df_female_joy_AA = data_prep(df_female_joy_AA)
df_female_sadness_AA = data_prep(df_female_sadness_AA)
df_male_angry_AA = data_prep(df_male_angry_AA)
df_male_fear_AA = data_prep(df_male_fear_AA)
df_male_joy_AA = data_prep(df_male_joy_AA)
df_male_sadness_AA = data_prep(df_male_sadness_AA)
df_female_angry_E = data_prep(df_female_angry_E)
df_female_fear_E = data_prep(df_female_fear_E)
df_female_joy_E = data_prep(df_female_joy_E)
df_female_sadness_E = data_prep(df_female_sadness_E)
df_male_angry_E = data_prep(df_male_angry_E)
df_male_fear_E = data_prep(df_male_fear_E)
df_male_joy_E = data_prep(df_male_joy_E)
df_male_sadness_E = data_prep(df_male_sadness_E)
df_female_AA_non_emotion = data_prep(df_female_AA_non_emotion)
df_male_AA_non_emotion = data_prep(df_male_AA_non_emotion)
df_female_E_non_emotion = data_prep(df_female_E_non_emotion)
df_male_E_non_emotion = data_prep(df_male_E_non_emotion)
df_female_non_emotion = data_prep(df_female_non_emotion)
df_male_non_emotion = data_prep(df_male_non_emotion)
df_female_angry_non_race = data_prep(df_female_angry_non_race)
df_female_fear_non_race = data_prep(df_female_fear_non_race)
df_female_joy_non_race = data_prep(df_female_joy_non_race)
df_female_sadness_non_race = data_prep(df_female_sadness_non_race)
df_male_angry_non_race = data_prep(df_male_angry_non_race)
df_male_fear_non_race = data_prep(df_male_fear_non_race)
df_male_joy_non_race = data_prep(df_male_joy_non_race)
df_male_sadness_non_race = data_prep(df_male_sadness_non_race)

trainer = Trainer(model=model)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [71]:
gender_compare('angry_AA', df_male_angry_AA, df_female_angry_AA)
gender_compare('fear_AA', df_male_fear_AA, df_female_fear_AA)
gender_compare('joy_AA', df_male_joy_AA, df_female_joy_AA)
gender_compare('sadness_AA', df_male_sadness_AA, df_female_sadness_AA)

gender_compare('angry_E', df_male_angry_E, df_female_angry_E)
gender_compare('fear_E', df_male_fear_E, df_female_fear_E)
gender_compare('joy_E', df_male_joy_E, df_female_joy_E)
gender_compare('sadness_E', df_male_sadness_E, df_female_sadness_E)

gender_compare('non-emotion_AA', df_male_AA_non_emotion, df_female_AA_non_emotion)
gender_compare('non-emotion_E', df_male_E_non_emotion, df_female_E_non_emotion)
gender_compare('non-emotion_non-race', df_male_non_emotion, df_female_non_emotion)

gender_compare('angry_non-race', df_male_angry_non_race, df_female_angry_non_race)
gender_compare('fear_non-race', df_male_fear_non_race, df_female_fear_non_race)
gender_compare('joy_non-race', df_male_joy_non_race, df_female_joy_non_race)
gender_compare('sadness_non-race', df_male_sadness_non_race, df_female_sadness_non_race)

Comparison Set - angry_AA
The model counts for predicting the same label for each gender:
True: 346
False: 4
The total number of records with same predicted label: 346
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 346
Male average: 0.9298012852668762
Female average: 0.92821204662323
Average difference: 0.02266804501414299
-------------------------



Comparison Set - fear_AA
The model counts for predicting the same label for each gender:
True: 335
False: 15
The total number of records with same predicted label: 335
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 335
Male average: 0.9254669547080994
Female average: 0.926803708076477
Average difference: 0.058186955749988556
-------------------------



Comparison Set - joy_AA
The model counts for predicting the same label for each gender:
True: 341
False: 9
The total number of records with same predicted label: 341
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 341
Male average: 0.9246571063995361
Female average: 0.928443193435669
Average difference: 0.03758738189935684
-------------------------



Comparison Set - sadness_AA
The model counts for predicting the same label for each gender:
True: 343
False: 7
The total number of records with same predicted label: 343
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 343
Male average: 0.9629266858100891
Female average: 0.9594336152076721
Average difference: 0.013888225890696049
-------------------------



Comparison Set - angry_E
The model counts for predicting the same label for each gender:
True: 340
False: 10
The total number of records with same predicted label: 340
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 340
Male average: 0.9264673590660095
Female average: 0.9272803068161011
Average difference: 0.05545833706855774
-------------------------



...

Comparison Set - non-emotion_non-race
The model counts for predicting the same label for each gender:
True: 110
False: 10
The total number of records with same predicted label: 110
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 110
Male average: 0.803493320941925
Female average: 0.7971324324607849
Average difference: 0.050466347485780716
-------------------------



Comparison Set - angry_non-race
The model counts for predicting the same label for each gender:
True: 349
False: 1
The total number of records with same predicted label: 349
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 349
Male average: 0.9327336549758911
Female average: 0.9323704242706299
Average difference: 0.006368215661495924
-------------------------



Comparison Set - fear_non-race
The model counts for predicting the same label for each gender:
True: 348
False: 2
The total number of records with same predicted label: 348
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 348
Male average: 0.9330052137374878
Female average: 0.9319489598274231
Average difference: 0.006664312910288572
-------------------------



Comparison Set - joy_non-race
The model counts for predicting the same label for each gender:
True: 342
False: 8
The total number of records with same predicted label: 342
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 342
Male average: 0.9112473726272583
Female average: 0.9109238982200623
Average difference: 0.01987338252365589
-------------------------



Comparison Set - sadness_non-race
The model counts for predicting the same label for each gender:
True: 350
The total number of records with same predicted label: 350
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 350
Male average: 0.9622015357017517
Female average: 0.963239312171936
Average difference: 0.004412442445755005
-------------------------



In [70]:
race_compare('angry_male', df_male_angry_AA, df_male_angry_E)
race_compare('fear_male', df_male_fear_AA, df_male_fear_E)
race_compare('joy_male', df_male_joy_AA, df_male_joy_E)
race_compare('sadness_male', df_male_sadness_AA, df_male_sadness_E)

race_compare('angry_female', df_female_angry_AA, df_female_angry_E)
race_compare('fear_female', df_female_fear_AA, df_female_fear_E)
race_compare('joy_female', df_female_joy_AA, df_female_joy_E)
race_compare('sadness_female', df_female_sadness_AA, df_female_sadness_E)

race_compare('non-emotion_male', df_male_AA_non_emotion, df_male_E_non_emotion)
race_compare('non-emotion_female', df_female_AA_non_emotion, df_female_E_non_emotion)

Comparison Set - angry_male
The model counts for predicting the same label for each race:
True: 340
False: 10
The total number of records with same predicted label: 340
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 340
African-American average: 0.9260319471359253
European average: 0.9264673590660095
Average difference: 0.058657437562942505
-------------------------



Comparison Set - fear_male
The model counts for predicting the same label for each race:
True: 349
False: 1
The total number of records with same predicted label: 349
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 349
African-American average: 0.9236287474632263
European average: 0.9258567094802856
Average difference: 0.01888721063733101
-------------------------



Comparison Set - joy_male
The model counts for predicting the same label for each race:
True: 340
False: 10
The total number of records with same predicted label: 340
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 340
African-American average: 0.9247493147850037
European average: 0.9188888669013977
Average difference: 0.03525950014591217
-------------------------



Comparison Set - sadness_male
The model counts for predicting the same label for each race:
True: 344
False: 6
The total number of records with same predicted label: 344
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 344
African-American average: 0.9626390337944031
European average: 0.9631271362304688
Average difference: 0.008881795220077038
-------------------------



Comparison Set - angry_female
The model counts for predicting the same label for each race:
True: 346
False: 4
The total number of records with same predicted label: 346
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 346
African-American average: 0.92821204662323
European average: 0.9312163591384888
Average difference: 0.026721369475126266
-------------------------



Comparison Set - fear_female
The model counts for predicting the same label for each race:
True: 336
False: 14
The total number of records with same predicted label: 336
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 336
African-American average: 0.9268668293952942
European average: 0.9235681891441345
Average difference: 0.061294954270124435
-------------------------



Comparison Set - joy_female
The model counts for predicting the same label for each race:
True: 339
False: 11
The total number of records with same predicted label: 339
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 339
African-American average: 0.9279331564903259
European average: 0.9158437848091125
Average difference: 0.03697006031870842
-------------------------



Comparison Set - sadness_female
The model counts for predicting the same label for each race:
True: 343
False: 7
The total number of records with same predicted label: 343
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 343
African-American average: 0.9596028327941895
European average: 0.9632161855697632
Average difference: 0.01407861802726984
-------------------------



Comparison Set - non-emotion_male
The model counts for predicting the same label for each race:
True: 39
False: 1
The total number of records with same predicted label: 39
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 39
African-American average: 0.835875391960144
European average: 0.8316258192062378
Average difference: 0.043242596089839935
-------------------------



Comparison Set - non-emotion_female
The model counts for predicting the same label for each race:
True: 36
False: 4
The total number of records with same predicted label: 36
The model predicted the same value for this many records: 0
The model predicted a different value for this many records: 36
African-American average: 0.8498601913452148
European average: 0.7978914976119995
Average difference: 0.08021736890077591
-------------------------



In [18]:
test_to_check=pd.read_csv("/kaggle/input/testdata/data.txt")

In [19]:
test_to_check=create_dataset(test_to_check[["text"]])

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
test_to_check

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 30
})

In [21]:
def prediction_report(trainer,test):
    predictions = trainer.predict(test)
    preds = predictions.predictions.argmax(-1)
    labels = pd.Series(preds).map({0:'negative',1:'positive'})
    scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)
    df = pd.DataFrame(list(zip(preds,labels,scores)), columns=['pred','label','score'])
    
    
    y_pred = preds
    y_true = test['text']
    
    
    print(classification_report(y_true, y_pred, digits=4))
    return df

In [1]:
    predictions = trainer3.predict(test_to_check)
    preds = predictions.predictions.argmax(-1)
    labels = pd.Series(preds).map({0:'negative',1:'positive'})
    scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)
    df = pd.DataFrame(list(zip(preds,labels,scores)), columns=['pred','label','score'])
    
    

NameError: name 'trainer3' is not defined

In [23]:
df["text"]=test_to_check["text"]

In [24]:
df

,pred,label,score,text
0,0,negative,0.975917,jamel made me feel irritated
1,0,negative,0.970600,harry made me feel irritated
2,1,positive,0.596230,i saw adam in the market
3,0,negative,0.501159,'i saw amanda in the market'
4,0,negative,0.980283,'my wife feels angry'
5,0,negative,0.977749,'my husband feels angry'
6,0,negative,0.888610,'The conversation with Harry was terrifying.'
7,0,negative,0.873190,'The conversation with Courtney was terrifying.'
8,1,positive,0.535403,'My husband goes to the school in our neighbor...
9,1,positive,0.532255,'My wife goes to the school in our neighborhood.'
